In [541]:
import pandas as pd
import nltk 
import re 
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder 
from nltk.tokenize import word_tokenize,WordPunctTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.models import load_model 
from tensorflow.keras.layers import Add,Dense, Flatten
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier 
from tensorflow.keras.models import Model

In [542]:
enc = OneHotEncoder() 

train_data = pd.read_excel("/kaggle/input/nlp-dataaa/train.xlsx")

train_data.info()  

one_hot = enc.fit_transform(train_data[['rating']]).toarray()

print('hekko',one_hot)
print(one_hot.shape)

X_train = train_data.drop('rating', axis=1) 
Y_train =  one_hot
#Y_train = train_data[['rating']] 

print(X_train)
print(Y_train)
print(Y_train.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32036 entries, 0 to 32035
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_description  32036 non-null  object
 1   rating              32036 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 500.7+ KB
hekko [[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
(32036, 3)
                                      review_description
0      شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...
1      خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...
2      تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...
3      فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...
4      سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...
...                                                  ...
32031  التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...
32032                                         y love you
32033      الباقه بتخلص وبشحن مرتين باقه

# Preprocessing

In [543]:
search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','"','?','؟','!']
replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
def process_review(review):
    processed_review = re.sub(r"[^\w\s]", '', review)# accept all things accept any word,digit,white space and replace with empty string
    processed_review = re.sub(r"[a-zA-Z]", '', processed_review) # here I remove all english lettters
    for i in range(0, len(search)):
        processed_review = processed_review.replace(search[i], replace[i])
    processed_review = re.sub(r"\d+", '', processed_review)
    processed_review = re.sub(r"\n", '', processed_review)# here remove new line
    processed_review = re.sub(r"\s+", ' ', processed_review)# here remove white spaces
    processed_review = re.sub(r'(.)\1+', r'\1', processed_review)# here if there are any repeate in any character replace it with one occurance
    
    return processed_review.strip() # return review after precessed without any additional white space

processed_reviews=[]
ratings=[]
reviews=X_train.values
rating=Y_train
print(Y_train)
print(rating)
print(len(reviews))
print(reviews[15127][0])
for i in range(len(reviews)):
    x=process_review(reviews[i][0])
    y=rating[i]
    processed_reviews.append(x)
    ratings.append(y)
print(processed_reviews[15127])
processed_reviews = np.array(processed_reviews)


[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
32036
ساره   اشهري
ساره اشهري


In [544]:
print(processed_reviews[:5])

['شركه زباله وسواقين بتبرشم ومفيش حتي رقم لشكاوي السواق يسيبك ويمشي وميرضاش يقف ويقولك اعملك ايه اتصل بالشركه وقولهم مرضيش يقفلي رغم اني حاجز من قبلها بيوم'
 'خدمه الدفع عن طريق الكي نت توقفت عندي اصبح فقط الدفع نقدا'
 'تطبيق غبي وجاري حذفه عاملين اكواد خصم ولما نستخدمها اكتر من مره يعملو بلوك لهاتف بالكامل ولما تواصل مع خدمه العملا يقولولك المشكله ملهاش حل هما الخسرانين وانصح الجميع انه ينزل تطبيقات المطاعم المشهوره زي كنتاكي او ماكدونالدز ويطلب منها افضل وبيكون في خصومات اقوي عليها وكل الوجبات متاحه'
 'فعلا تطبيق متاز بس لو في امكانيه يتيح لمستخدم التطبيق ان يستطيع تغير كلمه السر بدون الرجوع الي خدمه العملاء'
 'سيء جدا اسعار رسوم التوصيل لا تمت لواقع ب صله']


In [545]:
tokenizer = WordPunctTokenizer()
X_tokenized = [word_tokenize(sentence.lower()) for sentence in processed_reviews]

In [546]:
print(X_tokenized[:5])

[['شركه', 'زباله', 'وسواقين', 'بتبرشم', 'ومفيش', 'حتي', 'رقم', 'لشكاوي', 'السواق', 'يسيبك', 'ويمشي', 'وميرضاش', 'يقف', 'ويقولك', 'اعملك', 'ايه', 'اتصل', 'بالشركه', 'وقولهم', 'مرضيش', 'يقفلي', 'رغم', 'اني', 'حاجز', 'من', 'قبلها', 'بيوم'], ['خدمه', 'الدفع', 'عن', 'طريق', 'الكي', 'نت', 'توقفت', 'عندي', 'اصبح', 'فقط', 'الدفع', 'نقدا'], ['تطبيق', 'غبي', 'وجاري', 'حذفه', 'عاملين', 'اكواد', 'خصم', 'ولما', 'نستخدمها', 'اكتر', 'من', 'مره', 'يعملو', 'بلوك', 'لهاتف', 'بالكامل', 'ولما', 'تواصل', 'مع', 'خدمه', 'العملا', 'يقولولك', 'المشكله', 'ملهاش', 'حل', 'هما', 'الخسرانين', 'وانصح', 'الجميع', 'انه', 'ينزل', 'تطبيقات', 'المطاعم', 'المشهوره', 'زي', 'كنتاكي', 'او', 'ماكدونالدز', 'ويطلب', 'منها', 'افضل', 'وبيكون', 'في', 'خصومات', 'اقوي', 'عليها', 'وكل', 'الوجبات', 'متاحه'], ['فعلا', 'تطبيق', 'متاز', 'بس', 'لو', 'في', 'امكانيه', 'يتيح', 'لمستخدم', 'التطبيق', 'ان', 'يستطيع', 'تغير', 'كلمه', 'السر', 'بدون', 'الرجوع', 'الي', 'خدمه', 'العملاء'], ['سيء', 'جدا', 'اسعار', 'رسوم', 'التوصيل', 'لا', 'تمت', 'لوا

In [547]:
ar_stopwords = '''
أنفسنا مثل حيث ذلك بشكل لدى ألا عن إلي ب لنا وقالت فقط الذي الذى ا هذا غير أكثر اي أنا أنت ايضا اذا كيف وكل أو اكثر أي أن منه وكان وفي تلك إن سوف حين نفسها هكذا قبل حول منذ هنا عندما على ضمن لكن فيه عليه قليل صباحا لهم بان يكون بأن أما هناك مع فوق بسبب ما لا هذه و فيها ف ولم ل آخر ثانية انه من الان جدا به بن بعض حاليا بها هم أ كانت هي لها نحن تم أنفسهم ينبغي إلى فان وقد تحت' عند وجود الى فأن الي او قد خارج إنه اى مرة هؤلاء أنها إذا فهي فهى كل يمكن جميع أنفسكم فعل كان ثم لي الآن وقال فى في ديك لم لن له تكون الذين ليس التى التي أنه وان بعد حتى ان دون وأن لماذا يجري كلا إنها لك ضد وإن فهو انها منها أى لديه ولا بين خلال وما اما عليها بعيدا كما نفسي نحو هو نفسك نفسه انت ولن إضافي لقاء وكانت هى فما أيضا إلا معظم ومن إما الا بينما وهي وهو وهى و انا وانا أنا حتى و ايه بس
'''

ar_stopwords = word_tokenize(ar_stopwords)
print("length of stopwords is: ",len(ar_stopwords))
print(ar_stopwords)


length of stopwords is:  187
['أنفسنا', 'مثل', 'حيث', 'ذلك', 'بشكل', 'لدى', 'ألا', 'عن', 'إلي', 'ب', 'لنا', 'وقالت', 'فقط', 'الذي', 'الذى', 'ا', 'هذا', 'غير', 'أكثر', 'اي', 'أنا', 'أنت', 'ايضا', 'اذا', 'كيف', 'وكل', 'أو', 'اكثر', 'أي', 'أن', 'منه', 'وكان', 'وفي', 'تلك', 'إن', 'سوف', 'حين', 'نفسها', 'هكذا', 'قبل', 'حول', 'منذ', 'هنا', 'عندما', 'على', 'ضمن', 'لكن', 'فيه', 'عليه', 'قليل', 'صباحا', 'لهم', 'بان', 'يكون', 'بأن', 'أما', 'هناك', 'مع', 'فوق', 'بسبب', 'ما', 'لا', 'هذه', 'و', 'فيها', 'ف', 'ولم', 'ل', 'آخر', 'ثانية', 'انه', 'من', 'الان', 'جدا', 'به', 'بن', 'بعض', 'حاليا', 'بها', 'هم', 'أ', 'كانت', 'هي', 'لها', 'نحن', 'تم', 'أنفسهم', 'ينبغي', 'إلى', 'فان', 'وقد', 'تحت', "'", 'عند', 'وجود', 'الى', 'فأن', 'الي', 'او', 'قد', 'خارج', 'إنه', 'اى', 'مرة', 'هؤلاء', 'أنها', 'إذا', 'فهي', 'فهى', 'كل', 'يمكن', 'جميع', 'أنفسكم', 'فعل', 'كان', 'ثم', 'لي', 'الآن', 'وقال', 'فى', 'في', 'ديك', 'لم', 'لن', 'له', 'تكون', 'الذين', 'ليس', 'التى', 'التي', 'أنه', 'وان', 'بعد', 'حتى', 'ان', 'دون', 'وأن',

In [548]:
print(processed_reviews[1])

خدمه الدفع عن طريق الكي نت توقفت عندي اصبح فقط الدفع نقدا


In [549]:
# there are alot of stop words I can't write all of them can I use stop words in corpus of nltk
def remove_stop_words(text):
    filtered_words = [word for word in text if word not in ar_stopwords]
    filtered_text = ' '.join(filtered_words)
    return filtered_words

reviews_after_stopWords=[]
for i in range(len(X_tokenized)):
    filtered_review = remove_stop_words(X_tokenized[i])
    reviews_after_stopWords.append(filtered_review)
print(reviews_after_stopWords[1])
print(len(reviews_after_stopWords))

['خدمه', 'الدفع', 'طريق', 'الكي', 'نت', 'توقفت', 'عندي', 'اصبح', 'الدفع', 'نقدا']
32036


In [550]:
print(reviews_after_stopWords[:5])

[['شركه', 'زباله', 'وسواقين', 'بتبرشم', 'ومفيش', 'حتي', 'رقم', 'لشكاوي', 'السواق', 'يسيبك', 'ويمشي', 'وميرضاش', 'يقف', 'ويقولك', 'اعملك', 'اتصل', 'بالشركه', 'وقولهم', 'مرضيش', 'يقفلي', 'رغم', 'اني', 'حاجز', 'قبلها', 'بيوم'], ['خدمه', 'الدفع', 'طريق', 'الكي', 'نت', 'توقفت', 'عندي', 'اصبح', 'الدفع', 'نقدا'], ['تطبيق', 'غبي', 'وجاري', 'حذفه', 'عاملين', 'اكواد', 'خصم', 'ولما', 'نستخدمها', 'اكتر', 'مره', 'يعملو', 'بلوك', 'لهاتف', 'بالكامل', 'ولما', 'تواصل', 'خدمه', 'العملا', 'يقولولك', 'المشكله', 'ملهاش', 'حل', 'هما', 'الخسرانين', 'وانصح', 'الجميع', 'ينزل', 'تطبيقات', 'المطاعم', 'المشهوره', 'زي', 'كنتاكي', 'ماكدونالدز', 'ويطلب', 'افضل', 'وبيكون', 'خصومات', 'اقوي', 'الوجبات', 'متاحه'], ['فعلا', 'تطبيق', 'متاز', 'لو', 'امكانيه', 'يتيح', 'لمستخدم', 'التطبيق', 'يستطيع', 'تغير', 'كلمه', 'السر', 'بدون', 'الرجوع', 'خدمه', 'العملاء'], ['سيء', 'اسعار', 'رسوم', 'التوصيل', 'تمت', 'لواقع', 'صله']]


In [551]:
vocab_size = 500
embedding_dim=16
max_length = 16
trunc_type= 'post'
oov_tok=""

In [552]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(reviews_after_stopWords)
word_index = tokenizer.word_index


In [553]:

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train, X_test, label_train, label_test = train_test_split(reviews_after_stopWords, ratings, test_size=0.4, random_state=42)
X_valid, X_test, label_valid, label_test = train_test_split(X_train, label_train, test_size=0.5, random_state=42)
print("Training:", len(X_train))
print("Training:", len(label_train))
print("Testing: ", len(X_test))
print("Testing:", len(label_test))



Training: 19221
Training: 19221
Testing:  9611
Testing: 9611


In [554]:
# train
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating = trunc_type)
# validation   
valid_sequences = tokenizer.texts_to_sequences(X_valid)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length)
# test
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)
print(len(testing_padded))

9611


In [555]:
print(sequences[:5])

[[64, 1, 8, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 3, 265, 1, 1, 1, 1, 282, 1, 60, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [57, 137, 1, 1, 1, 1, 1, 1, 85, 1, 1, 1, 293, 1, 234, 1, 1, 491, 1, 411, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 391, 1, 1, 1, 1, 1, 1, 1, 1, 293, 1, 1, 1, 35, 1, 1, 301, 293, 1, 1, 1, 4, 1, 1, 1, 1, 234, 56, 120, 1, 137, 1, 4, 1, 1, 1, 459, 14, 1, 293, 1, 1, 1, 1, 1], [1, 1, 1, 1], [2, 54, 1, 1, 1, 1, 485, 1, 1, 1, 1, 1, 1, 1, 1]]


# Models

In [556]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(33373, 16,
                             input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='sigmoid')
])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 16, 16)            533968    
                                                                 
 bidirectional_12 (Bidirect  (None, 64)                9600      
 ional)                                                          
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 3)                 195       
                                                                 
Total params: 543763 (2.07 MB)
Trainable params: 543763 (2.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [557]:
num_epochs=5
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [558]:
print(type(padded))
print(type(label_train))
print(type(valid_padded))
print(type(label_valid))

<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'list'>


In [559]:
padded = np.array(padded)
valid_padded = np.array(valid_padded)
label_train = np.array(label_train)
label_valid = np.array(label_valid)
label_test = np.array(label_test)
testing_padded = np.array(testing_padded)

In [560]:
print(type(padded))
print(type(label_train))
print(type(valid_padded))
print(type(label_valid))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [561]:
print((padded.shape))
print((model.output).shape)
print(label_valid.shape)
print(valid_padded.shape)

(19221, 16)
(None, 3)
(9610, 3)
(9610, 16)


In [562]:
history=model.fit(padded, label_train, epochs=num_epochs,validation_data = (valid_padded,label_valid))
model.save("/kaggle/working/GRU.h5") 
#loaded_model = load_model("/kaggle/working/GRU.h5") 
#loss, accuracy = loaded_model.evaluate(test_data, test_targets) 

Epoch 1/5
601/601 [==============================] - 17s 20ms/step - loss: 0.4058 - accuracy: 0.7296 - val_loss: 0.3183 - val_accuracy: 0.8061
Epoch 2/5
601/601 [==============================] - 11s 18ms/step - loss: 0.3218 - accuracy: 0.8038 - val_loss: 0.3091 - val_accuracy: 0.8073
Epoch 3/5
601/601 [==============================] - 11s 19ms/step - loss: 0.3138 - accuracy: 0.8058 - val_loss: 0.3084 - val_accuracy: 0.8034
Epoch 4/5
601/601 [==============================] - 11s 18ms/step - loss: 0.3091 - accuracy: 0.8088 - val_loss: 0.3009 - val_accuracy: 0.8072
Epoch 5/5
601/601 [==============================] - 11s 18ms/step - loss: 0.3054 - accuracy: 0.8103 - val_loss: 0.3012 - val_accuracy: 0.8074


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [563]:
predictions = model.predict(testing_padded)

301/301 [==============================] - 2s 4ms/step


In [564]:
print(predictions)

[[0.82222396 0.0658035  0.13128215]
 [0.83368206 0.04529822 0.11052702]
 [0.0784376  0.07421966 0.86556613]
 ...
 [0.40594125 0.0852586  0.52033985]
 [0.28848433 0.06493538 0.6607097 ]
 [0.02061655 0.02290162 0.95830214]]


In [565]:
train_gru_results = model.evaluate(testing_padded, np.asarray(label_test))

301/301 [==============================] - 1s 4ms/step - loss: 0.3036 - accuracy: 0.8056


In [566]:
print(predictions)

[[0.82222396 0.0658035  0.13128215]
 [0.83368206 0.04529822 0.11052702]
 [0.0784376  0.07421966 0.86556613]
 ...
 [0.40594125 0.0852586  0.52033985]
 [0.28848433 0.06493538 0.6607097 ]
 [0.02061655 0.02290162 0.95830214]]


In [567]:
print(label_test)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [568]:
for i in predictions:
    max = 0
    indx = -1
    for j in range(len(i)):
        if i[j] > max:
            max = i[j]
            indx = j
            
    for k in range(len(i)):
        if i[k] == max:
            i[k] = 1
        else:
            i[k] = 0

In [569]:
ensemble_accuracy = accuracy_score(label_test, predictions)
print(f'Ensemble Model Accuracy: {ensemble_accuracy}')

Ensemble Model Accuracy: 0.8056393715534284


In [570]:
test_data=pd.read_csv("/kaggle/input/nlp-dataaa/test _no_label.csv")

x_test=test_data.drop('ID', axis=1) 

print(x_test)

                                    review_description
0    اهنئكم على خدمه العملاء في المحادثه المباشره م...
1    ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2      كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3                                              شغل طيب
4                                           بعد ماجربت
..                                                 ...
995                                              يستهل
996                             خدمة سيئة بكل المعايير
997                                      لؤي٠٣٣٢لؤ٣٤٣س
998                     تطبيق غير صادق ف خصم الكوبونات
999  تأخير + الموظفين غير مؤهلة .للأسف انا استخدم ط...

[1000 rows x 1 columns]


In [571]:
processed_reviews2=[]
reviews=x_test.values
print(len(reviews))
print(reviews[15][0])
for i in range(len(reviews)):
    x=process_review(reviews[i][0])
    processed_reviews2.append(x)

print(processed_reviews2[15])
processed_reviews2 = np.array(processed_reviews2)

1000
ني هتيعز
ني هتيعز


In [572]:
X_tokenized2 = [word_tokenize(sentence.lower()) for sentence in processed_reviews2]

In [573]:
reviews_after_stopWords2=[]
for i in range(len(X_tokenized2)):
    filtered_review2 = remove_stop_words(X_tokenized2[i])
    reviews_after_stopWords2.append(filtered_review2)
print(reviews_after_stopWords2[1])
print(len(reviews_after_stopWords2))

['متاز', 'ولكن', 'اتمني', 'المسابقات', 'والجوائز', 'طلبات', 'السعوديه', 'فسوف', 'مستخدمين', 'وافضل']
1000


In [574]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(reviews_after_stopWords2)
word_index = tokenizer.word_index

In [575]:
sequences = tokenizer.texts_to_sequences(reviews_after_stopWords2)
x_test_padded = pad_sequences(sequences, maxlen=max_length)
print(len(x_test_padded))
print(type(x_test_padded))

1000
<class 'numpy.ndarray'>


In [576]:
x_test_padded = np.array(x_test_padded)

In [577]:
output2 = model.predict(x_test_padded)

32/32 [==============================] - 0s 4ms/step


In [578]:
print(output2.shape)

(1000, 3)


In [579]:
print(output2)

[[0.02733237 0.02530252 0.94138455]
 [0.1262125  0.15232961 0.7918873 ]
 [0.7055609  0.076214   0.23674847]
 ...
 [0.28848433 0.06493538 0.6607097 ]
 [0.88823855 0.01158863 0.08661029]
 [0.17067152 0.09025748 0.79500073]]


In [580]:
print(output2)

[[0.02733237 0.02530252 0.94138455]
 [0.1262125  0.15232961 0.7918873 ]
 [0.7055609  0.076214   0.23674847]
 ...
 [0.28848433 0.06493538 0.6607097 ]
 [0.88823855 0.01158863 0.08661029]
 [0.17067152 0.09025748 0.79500073]]


In [581]:
print(len(output2))

1000


In [582]:
predictions2 = np.argmax(output2,axis=1)

In [583]:
res2 = []
for i in predictions2:
    if i == 0:
        x = -1
    elif i == 1:
        x = 0
    elif i == 2:
        x = 1
    res2.append(x)

In [584]:
print(res2)

[1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 

In [585]:
import csv

with open('output_predition.csv',mode='w')as out:
    field=['ID','rating']
    writer=csv.DictWriter(out,fieldnames=field)
    writer.writerow({'ID':'ID','rating':'rating'})
    for i in range(1,1001,1):
        writer.writerow({'ID':i,'rating':res2[i-1]})